In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [8]:
import torch 
import os
import pandas as pd
if torch.cuda.is_available():    
  device = torch.device("cuda")
  print('We will use the GPU:', torch.cuda.get_device_name(0))

We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!pip install -q kaggle 
!ls
if not os.path.exists('/root/.kaggle'):
  # !mkdir ~/.kaggle
  !mkdir /root/.kaggle
# !cp /content/kaggle.json ~/.kaggle/kaggle.json
!cp /content/gdrive/My\ Drive/kaggle.json ~/.kaggle/

gdrive	sample_data


In [5]:
!kaggle datasets download -d xhlulu/covidqa

100% 1.77G/1.78G [00:10<00:00, 181MB/s]
100% 1.78G/1.78G [00:10<00:00, 175MB/s]


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
# Unzip the dataset (if we haven't already)
if not os.path.exists('./covidqa/'):
    !unzip covidqa.zip

Archive:  covidqa.zip
  inflating: community.csv           
  inflating: electra-base-healthtap/__results__.html  
  inflating: electra-base-healthtap/custom.css  
  inflating: electra-base-healthtap/sigmoid.pickle  
  inflating: electra-base-healthtap/special_tokens_map.json  
  inflating: electra-base-healthtap/tokenizer_config.json  
  inflating: electra-base-healthtap/train_history.csv  
  inflating: electra-base-healthtap/transformer/config.json  
  inflating: electra-base-healthtap/transformer/tf_model.h5  
  inflating: electra-base-healthtap/vocab.txt  
  inflating: electra-base-stackexchange/__results__.html  
  inflating: electra-base-stackexchange/custom.css  
  inflating: electra-base-stackexchange/special_tokens_map.json  
  inflating: electra-base-stackexchange/tokenizer_config.json  
  inflating: electra-base-stackexchange/train_history.csv  
  inflating: electra-base-stackexchange/transformer/config.json  
  inflating: electra-base-stackexchange/transformer/sigmoid.pickl

In [8]:
news = pd.read_csv("./news.csv")
print(news.iloc[0]['question'])
news.head()

What are the symptoms?


,source,url,question,answer,wrong_answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What are the symptoms?,"Symptoms include fever, coughing, sore throat,...","As cases of coronavirus surge, health authorit..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,When should I get tested?,Your doctor will tell you if you need to get t...,If you have been in contact with a person with...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's the difference between physical distanc...,"As cases of coronavirus surge, health authorit...","Remember, the vast majority of Australians are..."
3,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,How do I practice physical distancing?,If you have been in contact with a person with...,Other countries have closed schools and univer...
4,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's closed?,Physical distancing is the reason the Federal ...,"Symptoms include fever, coughing, sore throat,..."


In [9]:
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 890kB 13.6MB/s 
     |████████████████████████████████| 1.0MB 16.1MB/s 
     |████████████████████████████████| 3.8MB 25.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=88c4a12d01bd6bb891c26e18e3f91a41dbb01d47d1da2abbae935013eaacbe6b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 0
qas = []
labels = []
for index, row in news.iterrows():
  correct = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['answer']).replace('\n', ' ') + ' [SEP]'
  incorrect = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['wrong_answer']).replace('\n', ' ') + ' [SEP]'
  correct_ids = tokenizer.encode(correct, add_special_tokens=False)
  incorrect_ids = tokenizer.encode(incorrect, add_special_tokens=False)
  if len(correct_ids) > 300 or len(incorrect_ids) > 300: 
    continue
  qas.append(correct)
  qas.append(incorrect)
  labels.append(1)
  labels.append(0)
  max_len = max(max_len, len(correct_ids), len(incorrect_ids))
for i in range(10):
  print("qa:", qas[i])
  print("label:", labels[i])
print(max_len)
print(len(qas))

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1370 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (688 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1362 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (714 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

qa: [CLS] What are the symptoms? [SEP] Symptoms include fever, coughing, sore throat, fatigue, and shortness of breath. However, be aware that at this stage if you have a cough or a cold, it's likely that you just have a cough or a cold and not coronavirus. If you have serious symptoms such as difficulty breathing, call 000 for urgent medical help. If you get these symptoms above after being in contact with someone who has been diagnosed with COVID-19, seek medical attention. The same goes if you develop symptoms within 14 days of returning home to Australia after being overseas. [SEP]
label: 1
qa: [CLS] What are the symptoms? [SEP] As cases of coronavirus surge, health authorities are advising Australians to practise physical distancing (also called social distancing) as much as possible to limit transmission. Physical distancing is what everyone can do - like staying home - regardless of whether or not they've been exposed to coronavirus. Self-isolating or self-quarantining is a more

In [0]:
# community = pd.read_csv('./community.csv')
# for index, row in community.iterrows():
#   correct = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['answer']).replace('\n', ' ') + ' [SEP]'
#   incorrect = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['wrong_answer']).replace('\n', ' ') + ' [SEP]'
#   correct_ids = tokenizer.encode(correct, add_special_tokens=False)
#   incorrect_ids = tokenizer.encode(incorrect, add_special_tokens=False)
#   if len(correct_ids) > 512 or len(incorrect_ids) > 512: 
#     continue
#   qas.append(correct)
#   qas.append(incorrect)
#   labels.append(1)
#   labels.append(0)
#   max_len = max(max_len, len(correct), len(incorrect))
# for i in range(10):
#   print("qa:", qas[i])
#   print("label:", labels[i])
# print(max_len)
# print(len(qas))

In [0]:
# multilingual = pd.read_csv('./multilingual.csv')
# for index, row in community.iterrows():
#   correct = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['answer']).replace('\n', ' ') + ' [SEP]'
#   incorrect = '[CLS] ' + str(row['question']).replace('\n', ' ') + ' [SEP] ' + str(row['wrong_answer']).replace('\n', ' ') + ' [SEP]'
#   correct_ids = tokenizer.encode(correct, add_special_tokens=False)
#   incorrect_ids = tokenizer.encode(incorrect, add_special_tokens=False)
#   if len(correct) > 512 or len(incorrect) > 512: 
#     continue
#   qas.append(correct_ids)
#   qas.append(incorrect_ids)
#   labels.append(1)
#   labels.append(0)
#   max_len = max(max_len, len(correct), len(incorrect))
# for i in range(10):
#   print("qa:", qas[i])
#   print("label:", labels[i])
# print(max_len)
# print(len(qas))

In [13]:
qas_ids = []
attention_masks = []
for qa in qas: 
  encoded_dict = tokenizer.encode_plus(
                        qa,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
  qas_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])
qas_ids = torch.cat(qas_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print(qas_ids[:10])
del qas
del news
# del community

tensor([[ 101, 2054, 2024,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ...,    0,    0,    0],
        [ 101, 2043, 2323,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2079,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ...,    0,    0,    0]])


In [14]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(qas_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
del qas_ids
del attention_masks

  700 training samples
   78 validation samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 4 #6 #16 

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
from transformers import BertForSequenceClassification, BertForQuestionAnswering, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 8

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 8 ========
Training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


  Batch    40  of    175.    Elapsed: 0:00:08.
  Batch    80  of    175.    Elapsed: 0:00:15.
  Batch   120  of    175.    Elapsed: 0:00:22.
  Batch   160  of    175.    Elapsed: 0:00:29.

  Average training loss: 0.70
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.44
  Validation Loss: 0.73
  Validation took: 0:00:01

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    175.    Elapsed: 0:00:07.
  Batch    80  of    175.    Elapsed: 0:00:14.
  Batch   120  of    175.    Elapsed: 0:00:21.
  Batch   160  of    175.    Elapsed: 0:00:28.

  Average training loss: 0.70
  Training epcoh took: 0:00:31

Running Validation...
  Accuracy: 0.60
  Validation Loss: 0.68
  Validation took: 0:00:01

======== Epoch 3 / 8 ========
Training...
  Batch    40  of    175.    Elapsed: 0:00:07.
  Batch    80  of    175.    Elapsed: 0:00:14.
  Batch   120  of    175.    Elapsed: 0:00:21.
  Batch   160  of    175.    Elapsed: 0:00:28.

  Average training loss: 0.70
  Training ep

In [22]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to ./model_save/


('./model_save/vocab.txt',
 './model_save/special_tokens_map.json',
 './model_save/added_tokens.json')

In [0]:
!cp -r ./model_save/ "./gdrive/My Drive/BERT fine-tuning/"

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp -r "./gdrive/My Drive/BERT fine-tuning/model_save" ./

In [11]:
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 13.0MB/s 
     |████████████████████████████████| 890kB 16.2MB/s 
     |████████████████████████████████| 3.8MB 16.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4b425109b0a0c3e534782da1ce985c662794fa68648b9980531fe62e951ffe5c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [14]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained("./model_save")
tokenizer = BertTokenizer.from_pretrained("./model_save")

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
!cp /content/gdrive/My\ Drive/JHUSpring2020/compute_seat.py ./

In [0]:
from compute_seat import SEAT, create_target_attribute_lists, A_words, B_words

with open('/content/gdrive/My Drive/JHUSpring2020/40_white_names.txt', 'r') as f:
    X_words = f.read().splitlines()
with open('/content/gdrive/My Drive/JHUSpring2020/40_black_names.txt', 'r') as f:
    Y_words = f.read().splitlines()

A, B, X, Y = create_target_attribute_lists(X_words, Y_words, A_words, B_words)
# seat = SEAT('./drive/My Drive/Final Projects/Deep Learning/BERT Fine-Tuning/HateSpeechDetection/model_save/')
seat = SEAT('./model_save/')
m, seat_score, effect_size = seat.run_seat(A, B, X, Y)


In [25]:
print(seat_score, effect_size)

0.0026663459846661963 1.0829589365189245


In [27]:
print(m)

[[0.9886965  0.99161546 0.98351306 0.99101868 0.98818851 0.97267885
  0.99176751 0.98886633 0.97406601 0.99130484 0.98833606 0.97700092
  0.9901699  0.98785348 0.97102508 0.99064955 0.98729073 0.97513819
  0.99162467 0.98993557 0.97464149 0.98869029 0.98595742 0.97699701
  0.98648359 0.98510328 0.97094154 0.99050567 0.99046604 0.97666525
  0.99069203 0.993074   0.97911873 0.99093196 0.98998454 0.97859187
  0.99134118 0.99043482 0.97937458 0.99143184 0.99162845 0.97853539
  0.99055855 0.98714749 0.97358379 0.9913519  0.98939668 0.97544329
  0.98702719 0.98585466 0.96769092 0.99033766 0.98934262 0.97652138]
 [0.97789356 0.98812166 0.98915397 0.98825928 0.98719722 0.9845606
  0.98825069 0.98708461 0.98486133 0.98549495 0.98668331 0.98687081
  0.98969774 0.98651524 0.98360672 0.98946184 0.98847158 0.98547487
  0.99015496 0.98918239 0.98584055 0.99071092 0.98893619 0.98614333
  0.9893831  0.98768741 0.98263485 0.98238041 0.98672071 0.98654188
  0.98090082 0.98582475 0.98739828 0.98430645 0.